In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

import sklearn
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import decomposition

In [ ]:
spark = SparkSession.builder.getOrCreate()
athlete_df_path = './ALL-ATHLETES.csv'
movie_path = "./movies.csv"

1. What is the difference between SparkContext and SparkSession?

Prior to Spark 2.0, SparkContext was used to be a channel to access all Spark functionalities. Spark driver programes use SparkContext to connect the cluster, and create RDDs, accumulators and broadcast variables on that cluster. Also, there are several SparkContexts, such as SQLContext. If we want to use them, we need to create SQLContext respectively.

SparkSession is a new entry point of Spark with the Dataset and DataFrame API. We just need to create a SparkSession, and then we can use the SparkSession to manipulate the data, such as query the data from a dataframe.

2. Explain the shufﬂe operation in Spark operations. Give an example.

There are partitions in RDD. When some kind of spark operations, such as join, groupByKey, or repartition, raise, the spark redistributes data to different groups or keys. It calls shuffle. For instance, if we want to repartition a dataframe by the a column, Spark will find all values of keys of the dataframe, and separate data into different partitions by the column.

In [ ]:
part1_q2_ex_df = spark.createDataFrame([10,11,13], "integer").toDF("age")
print("Before repartition: ")
print(part1_q2_ex_df.rdd.getNumPartitions())

repart_df = part1_q2_ex_df.repartition(2)
print("After repartition: ")
print(repart_df.rdd.getNumPartitions())

3. What is the difference between a Spark transformation and a Spark action.

When we use a spark transformation, we will produce a new RDD from existing RDD. 
If we use a saprk action, we will get non-RDD values, which are stored to the external storage system or dirver.

Transformation : union()

Action: count()

4. Cleaning Data

    a. What operations do we have on Panda DataFrames to deal with missing values?

    b. What are some of the problems that occur in dealing with missing values in Panda DataFrames?

    c. What operations do we have on Spark DataFrames to deal with missing values.

a. Panda DataFrames provides powerful funcions to deal with missing value such as isna(), notna(), notnull(), isnull(), and dropna(). isna(), notna(), notnull(), and isnull() will return list of boolean values which using indexes to indicate the result. Using the boolean list can revise missing values. For instance, 

"dropna()" can drop rows which contain missing values. Moreover, "dropna()" is able to set parameters to determine how to drop. For example, doprna(axis='columns') means drop every column which contains missing values.

b. A dataframe may have some of columns which contain NaN or None. If we use "dropna()" without setting parameters, then we will drop the rows which are correct. Furthermore, sometimes missing values are not NaN or None. Instead, using string to describe like few spaces, "none", or "empty". Since "dropna()" function is only able to detect NaN or None, a string which indicate none or nan cannot be detected. In this situation, after dropna() function, the missing values still exist.

c. Spark provides DataFrameNaFunctions which is in pyspark.sql.DataFrameNaFunctions. For example, na.drop(): return a dataframe which omits rows with null. na.fill(value): fill cells with the value.

5. What is a categorical variable? Give an example.

Categorical variable is a variable which contains names or labels to represent a category. Also, the categories must be a fixed or limited values. 
Example:
1. sexed in the class: Male and Female.
2. Programs which SDSU offers: 95 bachelor's, 78 masters and 22 doctoral degree programs.

6. What are hyperparameters? Give an example.

Hyperparameters are parameters which are setted before training. Every model has its hyperparameters. Hyperparameters may change the result after training or clustering.
Example:
n_clusters of KMeans  : Use a integer to set. A labels of data will cluster into the number which is setted.
eps of DBSCAN         : A programmer can determine the distance between two samples.

# Part Two: 
This part uses the athlete data set in the ﬁle ALL-ATHLETES.csv. This dataset contains information about athletes that participated in the London 2012 Olympics. 

1. The data requires some cleaning. Make sure that you clean the data before working on the rest of the problems. All solutions should start with the original dataset.

In [ ]:
all_athletes_df = pd.read_csv(athlete_df_path)
all_athletes_df = all_athletes_df.drop(columns=['Name','Date of birth', 'Place of birth','Event'])

2. Produce a scatter plot of the athletes weight verses height.

In [ ]:
athlete_with_height_weight_df = all_athletes_df.dropna(subset=['Weight', 'Height, cm'])
plot_x_weigth = athlete_with_height_weight_df['Weight']
plot_y_height = athlete_with_height_weight_df['Height, cm']

plt.figure(figsize=(20,6)) 
ax = plt.scatter(plot_x_weigth, plot_y_height); 
plt.xlabel('weight', fontsize = 18)
plt.ylabel('height', fontsize = 18)

3. Produce box plots of the following. How do the weights compare?
    
    a. Female weights
    
    b. Male weights
    
    c. Weight of the male metal winners
    
    d. Weight of the female metal winners

In [ ]:
athlete_with_weight_df = all_athletes_df.dropna(subset=['Weight'])

# plt.figure(figsize=(10,6)) 
athlete_male = athlete_with_weight_df.loc[athlete_with_weight_df["Sex"] =="M"]
athlete_won_metal_male = athlete_male.loc[athlete_with_weight_df["Total"] !=0]
athlete_won_metal_male = athlete_won_metal_male.replace("M","M metal winner")

athlete_female = athlete_with_weight_df.loc[athlete_with_weight_df["Sex"] =="F"]
athlete_won_metal_female = athlete_female.loc[athlete_with_weight_df["Total"] !=0]
athlete_won_metal_female = athlete_won_metal_female.replace("F","F metal winner")


athlete_df_for_box = pd.concat([athlete_female, athlete_male, \
                                athlete_won_metal_female, athlete_won_metal_male])
plt.figure(figsize=(20,6)) 
ax = sns.boxplot(x="Sex", y="Weight",data= athlete_df_for_box, orient='v')
ax.set_ylim([30, 200]) 
ax.set_title("Male")

the median of weight in female are a little higher than all of the female athletes. Male athletes have some result as well. Comparing with sex shows the median of weight of male is about 20 higher than female. Moreover, most of the athletes whose weight locate on outliers didn't win a medal.

4. Produce separate swarm plots of the ages of male metal winners and the female metal winners. How do the ages compare?

In [ ]:
athlete_with_weight_df = all_athletes_df.dropna(subset=['Weight'])

athlete_won_metal = athlete_with_height_weight_df.loc[athlete_with_height_weight_df["Total"] !=0]

plt.figure(figsize=(8,8))
swarmplot_metal = sns.swarmplot(x="Sex", y="Age",data= athlete_won_metal).set_title("Ages of male metal winners")
max_age = max(athlete_with_height_weight_df["Age"])
_ = plt.yticks(range(0, max_age+5,5)) 

There are the same conditions in female and male. Most of ages of metal winners are around 15~35. The youngest male metal winner is older than youngest female metal winner. Also, the oldest male metal winner is older than the oldest female metal winner.

5. Produce separate violin plots for the ages of the athletes in Archery, Sailing, and Swimming. How do the ages compare?

In [ ]:
athlete_age_sport_df = all_athletes_df.dropna(subset=['Age','Sport'])
athlete_age_sport_df = athlete_age_sport_df.loc[(athlete_age_sport_df["Sport"] == 'Archery') | (athlete_age_sport_df["Sport"] == 'Sailing') | (athlete_age_sport_df["Sport"] == 'Swimming')]
# athlete_won_metal

plt.figure(figsize=(10,15))
# plt.ylim(35, 160)
sns.violinplot(x="Sport", y="Age",data= athlete_age_sport_df)


The most of athletes in sailing whose ages are higher than the other two sports. Moreover, the youngest of athletes of sailing sport are older than the others. The range of ages of athletes in swimming are the narrowest of them. The Archery is widest. Archery has 10 to 50 years old athletes.It means a career of swimming is shorter than the others. Last but not least, the younger part of ages of per sport has more athletes than the older part. 

6. Produce a histogram of the metals won per country, sorted by the number of metals won.

In [ ]:
athlete_country_df = all_athletes_df.dropna(subset=['Country'])

athlete_country_df= athlete_country_df[["Country","Total"]].groupby("Country").sum().reset_index()
athlete_country_df = athlete_country_df.sort_values(by=['Total'], ascending =False)

plt.figure(figsize=(40,5))
bar_plot = sns.barplot(x="Country", y="Total", data=athlete_country_df)
bar_plot.set_title("Bar plot per country")
_ = bar_plot.set_xticklabels(labels=bar_plot.get_xticklabels(), rotation=90)

# Part Three: 
This part uses the movie dataset in the movies.csv ﬁle. The data set classiﬁes the movies as either a romance or an action movie. The columns other than "Title", "Genre", "Year", "Rating", "# Votes" and "# Words" in the ﬁle are all words that appear in some of the movies in our dataset. The words are stemmed. The column for a word shows the percentage of the total words in the movie were that particular word. So for example the word “the” was 0.043807463 percent of the words that were spoken in the movie “The Terminator”.

1. Using scikit-learn split the movie data into a training and test set. Create three different models from the training set to classify movies as action or romance using K-means, DBSCAN and GaussianNB.

In [ ]:
all_movies_df = pd.read_csv(movie_path)
all_movies_df = all_movies_df.drop(columns=["Rating","Title","Year", "# Votes", "# Words"])

columns_names = all_movies_df.columns
columns_without_genre = columns_names.drop( ["Genre"])

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(all_movies_df[columns_without_genre], all_movies_df["Genre"], test_size=0.2,random_state=34)

In [ ]:
#K-means
label_action = 0
label_romance = 0

kmeans = KMeans(n_clusters = 2,random_state = 1).fit(x_train)

romance_0 = 0
romance_1 = 0

ind = 0
for label in y_train:
    if label == "romance":
        if kmeans.labels_[ind] ==0:
            romance_0 += 1
        else:
            romance_1 += 1
    ind += 1

if romance_1 > romance_0:
    label_romance = 1
    label_action = 0
else:
    label_romance = 0
    label_action = 1
    
kmean_y_predict = kmeans.predict(x_test)

kmean_y_test_label = y_test.replace("romance", label_romance)
kmean_y_test_label = kmean_y_test_label.replace("action", label_action)

kmeans_result = pd.DataFrame({"Y test": kmean_y_test_label, "Y predict": kmean_y_predict})
kmeans_result

In [ ]:
# DASCAN
dbscan = DBSCAN(eps=0.05, min_samples=10).fit(x_train)

# dbscan_y_test_label = y_train.replace("romance", 0)
dbscan_y_train_label = y_train.replace("romance", 0)
dbscan_y_train_label = dbscan_y_train_label.replace("action", 1)
dbscan_y_train_label = dbscan_y_train_label.reset_index()
dbscan_y_train_label = dbscan_y_train_label.drop(columns=["index"])

y_predict = dbscan.labels_
remove_ind = []
dbscan_y_predict = []
for ind in range(y_predict.size):
    if(y_predict[ind] == -1):
        remove_ind.append(ind)
    else:
        dbscan_y_predict.append(y_predict[ind])

dbscan_y_train_label = dbscan_y_train_label.drop(index = remove_ind)
dbscan_y_train_label = dbscan_y_train_label.values.reshape(-1)

dbscan_result = pd.DataFrame({"Y test": dbscan_y_train_label, "Y predict": dbscan_y_predict})
dbscan_result

In [ ]:
#GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
gnb_y_predict = clf.predict(x_test)

gnb_result = pd.DataFrame({"Y test": y_test, "Y predict": gnb_y_predict})
gnb_result

2. Compare the performance of the three classiﬁers using accuracy score and confusion matrix. Which classiﬁer is better? Why?

In [ ]:
# confusion_matrix and accuracy
kmean_cm = metrics.confusion_matrix(kmean_y_test_label,kmean_y_predict)
kmean_accuracy = metrics.accuracy_score(kmean_y_test_label,kmean_y_predict)
print("K-Means CM          Accuracy")
print(kmean_cm, "          " , kmean_accuracy, end="\n\n")

dbscan_cm = metrics.confusion_matrix(dbscan_y_train_label,dbscan_y_predict)
dbscan_accuracy = metrics.accuracy_score(dbscan_y_train_label,dbscan_y_predict)
print("DBSCAN CM          Accuracy")
print(dbscan_cm, "          ",dbscan_accuracy, end="\n\n")

gnb_cm = metrics.confusion_matrix(y_test,gnb_y_predict)
gaussiannb_accuracy = metrics.accuracy_score(y_test,gnb_y_predict)
print("GaussianNB CM          Accuracy")
print(gnb_cm, "          ",gaussiannb_accuracy , end="\n\n")

According to the confusion matrices and accuracies, K-Means is the better than the others. Although, an accuracy of K-Means are not stable, the accuracy can up to 80%. DBSCAN is 45%, and Gaussian NB is 57%. Both of them are lower than K-Means. In my code, I set the random_state as 1 to make sure the result will be the same.

3. Compare two cluster results using Adjusted Rand index, Homogeneity, completeness and V-measure and Silhouette Coefﬁcient. 

In [ ]:
#K-Means Classifier
# 1. Adjusted Rand index
kmean_ari = metrics.adjusted_rand_score(kmean_y_test_label,kmean_y_predict)

# 2. Homogeneity, completeness, and V-measure
kmean_homo,kmean_complete,kmean_vmeasure = metrics.homogeneity_completeness_v_measure(kmean_y_test_label,kmean_y_predict)

# 3. Silhouette Coefficient 
kmean_silhouette = metrics.silhouette_score(x_test,kmean_y_predict)

print("ARI                : ", kmean_ari)
print("Homogeneity score  : ", kmean_homo)
print("Completeness score : ", kmean_complete)
print("V-measure score    : ", kmean_vmeasure)
print("Silhouette score   : ", kmean_silhouette)

In [ ]:
#DBSCN Classifier
# 1. Adjusted Rand index
dbscan_ari = metrics.adjusted_rand_score(dbscan_y_train_label,dbscan_y_predict)

# 2. Homogeneity, completeness, and V-measure
dbscan_homo,dbscan_complete,dbscan_vmeasure = metrics.homogeneity_completeness_v_measure(dbscan_y_train_label,dbscan_y_predict)

# 3. Silhouette Coefficient 
dbscan_silhouette = metrics.silhouette_score(x_train,dbscan.labels_)

print("ARI                : ", dbscan_ari)
print("Homogeneity score  : ", dbscan_homo)
print("Completeness score : ", dbscan_complete)
print("V-measure score    : ", dbscan_vmeasure)
print("Silhouette score   : ", dbscan_silhouette)

In [ ]:
#GaussianNB Classifier
# 1. Adjusted Rand index
gnb_ari = metrics.adjusted_rand_score(y_test,gnb_y_predict)

# 2. Homogeneity, completeness, and V-measure
gnb_homo,gnb_complete,gnb_vmeasure = metrics.homogeneity_completeness_v_measure(y_test,gnb_y_predict)

# 3. Silhouette Coefficient 
gnb_silhouette = metrics.silhouette_score(x_test,gnb_y_predict)

print("ARI                : ", gnb_ari)
print("Homogeneity score  : ", gnb_homo)
print("Completeness score : ", gnb_complete)
print("V-measure score    : ", gnb_vmeasure)
print("Silhouette score   : ", gnb_silhouette)

ARI:
ARI shows that two ways to cluster the dataset are both not strong enough to cluster or predict the result, since scores are low. The scrore of K-Means is around 0.3~0.4, the score of GaussianNB is around -0.001, and the score of DBSCAN is 0.
Having low scores with the same dataset in three classifiers tells us the dataset is not good enough to be a data to train or cluster.

Homogeneity:
Homogeneity score tells us that a cluster of prediction maps to differnt values a lot. It means that there isn't much difference between each movie. Therefore, the two different types of movies are high probability to cluster or predict to the same types. Although, we can figure out that K-Means is the best, the score is still low. Consequently, the result shows the movies of the dataset doesn't contain obvious attributes between each other.

Completeness:
Completeness is similar to homogeneity, but it's opposite to homogenity. The low score means that two same type of movies are high probability to cluster or predict to different types. The score of DBSCAN is 1, because it only one cluster, excluding noise. Therefore, DBSCAN indeed has perfect completeness, but it cannot determine DBSCAN is the best. The conclusion is the same as homogeneity, we need more data or  more information which has more difference between each movie to improve.  

V-measure: 
The V-measure is the harmonic mean between homogeneity and completeness. The conclusion is similar to both of them. Dataset needs more information.

Silhouette:
Low silhouette score expresses that points overlap a lot. That is, it's really hard to distinguish between each of point.
With our dataset, the score is low. It means the movies in our dataset are not very different from romance and action. The conclusion is consistent in the conclusion of other scores. 

4. The movie dataset has a lot of features. Use PCA to reduce the dataset to 30 independent variables. Repeat 1-3 on the reduced dataset. (PCA - Principle Component Analysis)

In [ ]:
pca = decomposition.PCA(n_components=30)
principalComponents = pca.fit_transform(all_movies_df[columns_without_genre])
principalDf = pd.DataFrame(data = principalComponents)
pca_x_train, pca_x_test, pca_y_train, pca_y_test = sklearn.model_selection.train_test_split(principalDf, all_movies_df["Genre"], test_size=0.2,random_state=34)

4.1.Create three different models from the training set to classify movies as action or romance using K-means, DBSCAN and GaussianNB.

In [ ]:
#K-means
label_action = 0
label_romance = 0

pca_kmeans = KMeans(n_clusters = 2,random_state = 8).fit(pca_x_train)

romance_0 = 0
romance_1 = 0

ind = 0
for label in pca_y_train:
    if label == "romance":
        if kmeans.labels_[ind] ==0:
            romance_0 += 1
        else:
            romance_1 += 1
    ind += 1

if romance_1 > romance_0:
    label_romance = 1
    label_action = 0
else:
    label_romance = 0
    label_action = 1
    
pca_kmean_y_predict = pca_kmeans.predict(pca_x_test)

pca_kmean_y_test_label = pca_y_test.replace("romance", label_romance)
pca_kmean_y_test_label = pca_kmean_y_test_label.replace("action", label_action)

pca_kmeans_result = pd.DataFrame({"Y test": pca_kmean_y_test_label, "Y predict": pca_kmean_y_predict})
pca_kmeans_result

In [ ]:
#PCA DASCAN
pca_dbscan = DBSCAN(eps=0.05, min_samples=10).fit(pca_x_train)

pca_dbscan_y_train_label = pca_y_train.replace("romance", 0)
pca_dbscan_y_train_label = pca_dbscan_y_train_label.replace("action", 1)
pca_dbscan_y_train_label = pca_dbscan_y_train_label.reset_index()
pca_dbscan_y_train_label = pca_dbscan_y_train_label.drop(columns=["index"])

pca_y_predict = pca_dbscan.labels_
remove_ind = []
pca_dbscan_y_predict = []
for ind in range(pca_y_predict.size):
    if(pca_y_predict[ind] == -1):
        remove_ind.append(ind)
    else:
        pca_dbscan_y_predict.append(pca_y_predict[ind])
        
pca_dbscan_y_train_label = pca_dbscan_y_train_label.drop(index = remove_ind)
pca_dbscan_y_train_label = pca_dbscan_y_train_label.values.reshape(-1)


pca_dbscan_result = pd.DataFrame({"Y test": pca_dbscan_y_train_label, "Y predict": pca_dbscan_y_predict})
pca_dbscan_result

In [ ]:
#PCA GaussianNB
pca_clf = GaussianNB()
pca_clf.fit(pca_x_train, pca_y_train)
pca_gnb_y_predict = pca_clf.predict(pca_x_test)

pca_gnb_result = pd.DataFrame({"Y test": y_test, "Y predict": pca_gnb_y_predict})
pca_gnb_result

4.2. Compare the performance of the three classiﬁers using accuracy score and confusion matrix. Which classiﬁer is better? Why?

In [ ]:
#PCA confusion_matrix and accuracy
pca_kmean_cm = metrics.confusion_matrix(pca_kmean_y_test_label,pca_kmean_y_predict)
pca_kmean_accuracy = metrics.accuracy_score(pca_kmean_y_test_label,pca_kmean_y_predict)
print("K-Means CM          Accuracy")
print(pca_kmean_cm, "          " , pca_kmean_accuracy, end="\n\n")

pca_dbscan_cm = metrics.confusion_matrix(pca_dbscan_y_train_label,pca_dbscan_y_predict)
pca_dbscan_accuracy = metrics.accuracy_score(pca_dbscan_y_train_label,pca_dbscan_y_predict)
print("DBSCAN CM          Accuracy")
print(pca_dbscan_cm, "          ",pca_dbscan_accuracy, end="\n\n")

pca_gnb_cm = metrics.confusion_matrix(y_test,pca_gnb_y_predict)
gaussiannb_accuracy = metrics.accuracy_score(y_test,pca_gnb_y_predict)
print("GaussianNB CM          Accuracy")
print(pca_gnb_cm, "          ",gaussiannb_accuracy , end="\n\n")

Using a dataset handled by pca improves the accruacies of three models. According the accuraies, the K-means is sill the best model of three. Its accuracy can be up to 83%, 20~40% more than the other two models. The accuracy up to 83% means it closes to be a well model for clustering. Using PCA helps us decrease the features of movies from 5000 to 30, and then reduce the effort of clustering or taining. Moreover, using dataset handled by pca improves the accruacies.

4.3 Compare two cluster  results using Adjusted Rand index, Homogeneity, completeness and V-measure and Silhouette Coefﬁcient. 

In [ ]:
#PCA K-Means Classifier
# 1. Adjusted Rand index
pca_kmean_ari = metrics.adjusted_rand_score(pca_kmean_y_test_label,pca_kmean_y_predict)

# 2. Homogeneity, completeness, and V-measure
pca_kmean_homo,pca_kmean_complete,pca_kmean_vmeasure = metrics.homogeneity_completeness_v_measure(pca_kmean_y_test_label,pca_kmean_y_predict)

# 3. Silhouette Coefficient 
pca_kmean_silhouette = metrics.silhouette_score(pca_x_test,pca_kmean_y_predict)

print("ARI                : ", pca_kmean_ari)
print("Homogeneity score  : ", pca_kmean_homo)
print("Completeness score : ", pca_kmean_complete)
print("V-measure score    : ", pca_kmean_vmeasure)
print("Silhouette score   : ", pca_kmean_silhouette)

In [ ]:
#PCA DBSCN Classifier
# 1. Adjusted Rand index
pca_dbscan_ari = metrics.adjusted_rand_score(pca_dbscan_y_train_label,pca_dbscan_y_predict)

# 2. Homogeneity, completeness, and V-measure
pca_dbscan_homo,pca_dbscan_complete,pca_dbscan_vmeasure = \
metrics.homogeneity_completeness_v_measure(pca_dbscan_y_train_label,pca_dbscan_y_predict)

# 3. Silhouette Coefficient 
pca_dbscan_silhouette = metrics.silhouette_score(pca_x_train,pca_dbscan.labels_)

print("ARI                : ", pca_dbscan_ari)
print("Homogeneity score  : ", pca_dbscan_homo)
print("Completeness score : ", pca_dbscan_complete)
print("V-measure score    : ", pca_dbscan_vmeasure)
print("Silhouette score   : ", pca_dbscan_silhouette)

In [ ]:
#PCA GaussianNB Classifier
# 1. Adjusted Rand index
pca_gnb_ari = metrics.adjusted_rand_score(y_test,pca_gnb_y_predict)

# 2. Homogeneity, completeness, and V-measure
pca_gnb_homo,pca_gnb_complete,pca_gnb_vmeasure = metrics.homogeneity_completeness_v_measure(y_test,pca_gnb_y_predict)

# 3. Silhouette Coefficient 
pca_gnb_silhouette = metrics.silhouette_score(pca_x_test,pca_gnb_y_predict)

print("ARI                : ", pca_gnb_ari)
print("Homogeneity score  : ", pca_gnb_homo)
print("Completeness score : ", pca_gnb_complete)
print("V-measure score    : ", pca_gnb_vmeasure)
print("Silhouette score   : ", pca_gnb_silhouette)

Using the dataset handled by pca increases most of the scores. Although most of scores have great raise, the scores are still low. Also, the Silhouette score shows the new dataset still haven't have obvious attributes to obesrve. We can find that the Silhouette score and completeness score of DBSCN are higher. Yet, it doesn't express DBSCAN is the best. Since DBSCAN only cluster data into one cluster and noise, this situation makes the scores error. Consequently, we learn that PCA is able to improve training or clustering. However, if a dataset doen't provide enough information or clear feature, then PCA can just upgrade models a little.

4.a. How do the result compare using the reduced dataset

The result is similar to the original dataset. The new scores are a little higher than original scores. Threfore, PCA may convert a dataset to better one which has more obvious features. The most important thing is a new dataset only contains 30 features. It can avoid using too much memory.  

4.b. How much of the variation of the original dataset is in the new 30 dimensions.

In [ ]:
sum_ratio = sum(pca.explained_variance_ratio_)
print("Sum of explained variance: ", "{0:.2%}".format(sum_ratio))

The new 30 dimensions contains around 72% of the variance, and the result upgrades a little. Thus, after dealed with by PCA, we may not consider 100% variance, but the result improves. 